In [1]:
# Justwatch url

justwatch_url = 'https://www.justwatch.com/uk'

In [20]:
!pip install lxml

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB 108.9 kB/s eta 0:00:35
   ---------------------------------------- 0.0/3.8 MB 145.2 kB/s eta 0:00:27
   ---------------------------------------- 0.0/3.8 MB 178.6 kB/s eta 0:00:22
    --------------------------------------- 0.1/3.8 MB 192.5 kB/s eta 0:00:20
    --------------------------------------- 0.1/3.8 MB 206.9 kB/s eta 0:00:19
    --------------------------------------- 0.1/3.8 MB 238.1 kB/s eta 0:00:16
   - -------------------------------------- 0.1/3.8 MB 288.1 kB/s eta 0:00:13
   - -------------------------------------- 0.2/3.8 MB 361.7 kB/s eta 0:00:11
   -- ------------------------------------- 0.2/3.8 MB 443.4 kB/s eta 0:00:09
   --- ------------------------------------ 0.3/3.8 MB 542.9 kB/s eta 0:00:07
   ---- -----

In [14]:
# Necessary imports
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
from urllib.parse import urljoin
import warnings
import pprint
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
warnings.filterwarnings("ignore")
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('headless')
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--incognito")
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

In [8]:
# Different provider url

netflix = '/provider/netflix'
disney_plus = '/provider/disney-plus' 
iplayer = '/provider/bbc-iplayer'
apple_tv = '/provider/apple-tv-plus'
amazon = '/provider/amazon-prime-video'
all_sources = [netflix, disney_plus, iplayer, apple_tv, amazon]

In [9]:
# Make Source URL

def get_url(source):
    return justwatch_url + source

In [18]:
# Install Driver
service = Service()
options = webdriver.ChromeOptions()
# options.add_argument('headless')
driver = webdriver.Chrome(service=service, options=options)
driver.maximize_window()

In [21]:
df = pd.DataFrame()
for source in all_sources[0:1]:
    data = {
            'Source': '',
            'Movie/Show': '',
            'Parameter Name': '',
            'Parameter Value': ''
            }
    print(f'Source name --> {source.split("/")[-1]}')
    print(f'Source URL --> {get_url(source)}')
    driver.get(get_url(source))
    time.sleep(1)
    # Scroll to bottom to load all movies/shows 
#     last_height = driver.execute_script("return document.body.scrollHeight")
#     while True:
#         # Scroll down to bottom
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         # Wait to load page
#         time.sleep(5)
#         # Calculate new scroll height and compare with last scroll height
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:
#             break
#         last_height = new_height
#         time.sleep(1)
#         soup = bs(driver.page_source, 'lxml')
#         all_movies = soup.findAll('div', class_='title-list-grid__item')
#         print(f'Number of all movies/shows found in {source.split("/")[-1]} ---> {len(all_movies)}')
    i  = 1
    while i < 10:
        print(i)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        i += 1
        time.sleep(3)
    time.sleep(0.5)
    soup = bs(driver.page_source, 'lxml')
    all_movies = soup.findAll('div', class_='title-list-grid__item')
    print(f'Number of all movies/shows found in {source.split("/")[-1]} ---> {len(all_movies)}')
    data['Source'] = source.split("/")[-1].title()
    all_movies_links = []
    
    
    for movie in all_movies:
        justwatch_movie_url = 'https://www.justwatch.com' + movie.find('a')['href']
        print(f"Movie Link from Justwatch for {source.split('/')[-1]} --> {justwatch_movie_url}")
        all_movies_links.append(justwatch_movie_url)
        res = requests.get(justwatch_movie_url)
        soup = bs(res.content, 'lxml')
        try:
            director = soup.find('div', text="Director").findNextSibling().text.strip()
        except:
            director = ''
        try:
            age_rating = soup.find('div', text="Age rating").findNextSibling().text.strip()
        except:
            age_rating = ''
        try:
            all_streaming_platforms = ', '.join([stream['title'] for stream in soup.find('div', class_='price-comparison__grid__row__holder').findAll('img')])
        except:
            all_streaming_platforms = ''
        try:
            genres = soup.find('div', text="Genres").findNextSibling().text.strip()
        except:
            genres = ''
        imdb_link = soup.find('div', {'v-uib-tooltip': 'IMDB'}).find('a')['href']
        print(f'IMBD links extracted from {source.split("/")[-1]} ---> {imdb_link}')
        driver.get(imdb_link)
        time.sleep(1)
        soup = bs(driver.page_source, 'lxml')
        
        ###
        try:
            title = soup.find('h1', class_='sc-b73cd867-0 cEmnhL').text.strip()
        except:
            try:
                title = soup.find('span', class_='sc-afe43def-1 fDTGTb').text.strip()
            except:
                title = 'Title NA'
        try:
            imdb_rating = soup.find('span', class_='sc-e457ee34-1 gvYTvP').text.strip()
        except:
            imdb_rating = ''
        try:
            episodes  = soup.find('div', {'data-testid': 'episodes-header'}).find('span', class_='ipc-title__subtext').text.strip()
        except:
            episodes = ''
        try:
            top_stars = ', '.join([star.text for star in soup.find('a', text='Stars').findNextSibling().findAll('a')])
        except:
            top_stars = ''
        try:
            top_casts = ', '.join([' - '.join([a.text for a in cast.findAll('a')]) for cast in soup.findAll('div', class_='sc-bfec09a1-7 dpBDvu')])
        except:
            top_casts = ''
        try:
            creator = soup.find('span', text='Creator').findNextSibling().text.strip()
        except: 
            try:
                creator = ', '.join([creator.text for creator in soup.find('span', text='Creators').findNextSibling().findAll('a')])
            except:
                creator = ''
        if genres == '':
            try:
                genres = ', '.join([genre.text for genre in soup.find('span', text='Genres').findNextSibling().findAll('a')])
            except:
                genres = ''
        try:
            release_date = soup.find('a', text='Release date').findNextSibling().text.split('(')[0].strip()
        except:
            release_date = ''
        try:
            release_country = soup.find('span', text='Country of origin').findNextSibling().text.strip()
        except:
            release_country  = ''
        try:
            language = soup.find('span', text='Language').findNextSibling().text.strip()
        except:
            language = ''
        try:
            production_company = soup.find('a', text='Production company').findNextSibling().text.strip()
        except:
            production_company = ''
        try:
            runtime = soup.find('span', text='Runtime').findNextSibling().text.strip()
        except:
            runtime = ''
        try:
            plot_summary = soup.find('div', {'data-testid': 'storyline-plot-summary'}).text.strip()
        except:
            plot_summary = ''
        try:
            similar_shows = ', '.join([show.text for show in soup.find('section', {'data-testid': 'MoreLikeThis'}).findAll('span', {'data-testid': 'title'})])
        except:
            similar_shows = ''
        data['Movie/Show'] = title
        data['Parameter Name'] = [
            'Director', 'Age Rating', 'All Streaming Platforms', 'IMDB Rating', 'Episodes', 
            'Top Stars', 'Top Casts', 'Creator', 'Genres', 'Release Date', 'Release Country', 
            'Language', 'Production Company', 'Runtime', 'Plot Summary', 'Similar Shows']
        data['Parameter Value'] = [director, age_rating, all_streaming_platforms, 
                                   imdb_rating, episodes, top_stars, top_casts, creator,
                                   genres, release_date, release_country, language, production_company, 
                                   runtime, plot_summary, similar_shows]
        print(data)
        temp_df = pd.DataFrame(data) # convert dictionary to dataframe
        df = df.append(temp_df, ignore_index=True) 

Source name --> netflix
Source URL --> https://www.justwatch.com/uk/provider/netflix
1
2
3
4
5
6
7
8
9


FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [15]:
soup = bs(driver.page_source, 'lxml')
all_movies = soup.findAll('div', class_='title-list-grid__item')
print(all_movies[:-1])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
df = df.set_index(['Source', 'Movie/Show', 'Parameter Name'])
df

Parameter Value
Source      Movie/Show      Parameter Name                                                            
Disney-Plus The Mandalorian Director                                                                  
                            Age Rating                                                              PG
                            All Streaming Platforms                                        Disney Plus
                            IMDB Rating                                                               
                            Episodes                                                                24
...                                                                                                ...
Bbc-Iplayer The Gold        Language                                                           English
                            Production Company                         ViacomCBS International Studios
                            Runtime                                                         58 minutes
                            Plot Summary                                                              
                            Similar Shows            Better, Happy Valley, Funny Woman, Stonehouse,...

[96 rows x 1 columns]

In [8]:
df.to_excel('temp.xlsx')